# 4.4 Review Playtime

In [93]:
import nltk
import numpy as np
import pandas as pd
import re
from sklearn.dummy import DummyRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import ElasticNet, Lasso, Ridge, SGDRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import RobustScaler

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
languages = [
    'arabic', 'azerbaijani', 'bengali', 'danish', 'dutch', 'english',
    'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian',
    'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian',
    'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish'
]
english_stopwords = set(stopwords.words('english'))
all_stopwords = set()
for language in languages:
    all_stopwords |= set(stopwords.words(language))

In [3]:
DIR_SAMPLE = '../data/samples/'
PATH_REVSENTS = DIR_SAMPLE + 'review_sentiment/%s.csv'

In [4]:
def preprocess_text(text, is_english=True):
    if is_english:
        text = text.lower()
        stopword_list = english_stopwords
    else:
        stopword_list = all_stopwords
    text = re.sub(r'[^\w\s]', ' ', text, re.UNICODE)
    text = ' '.join([
        word
        for word in text.split()
        if word not in stopword_list
    ])
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [57]:
def read_data(sample_name, train_size=0.8, seed=None):
    df = pd.read_csv(PATH_REVSENTS % sample_name)[['pt_review', 'text']].reset_index(drop=True)
    if sample_name.startswith('eng'):
        df['text'] = df['text'].apply(lambda text: preprocess_text(text, is_english=True))
    else:
        df['text'] = df['text'].apply(lambda text: preprocess_text(text, is_english=False))
    scaler = RobustScaler()
    df['pt_review_scaled'] = scaler.fit_transform(np.array(df['pt_review']).reshape(-1, 1))
    train, validation = train_test_split(df, train_size=train_size, random_state=seed)
    return train, validation

In [101]:
train, validation = read_data('eng_any_any_100000')
Xt, Yt = train['text'].tolist(), train['pt_review_scaled'].tolist()
Xv, Yv = validation['text'].tolist(), validation['pt_review_scaled'].tolist()

In [102]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), binary=True, smooth_idf=False)
Xt_tfidf = tfidf.fit_transform(Xt)
Xv_tfidf = tfidf.transform(Xv)

In [103]:
alphas = [1]#[0.1, 1, 10]
for alpha in alphas:
    bl_model1 = DummyRegressor(strategy='mean')
    #bl_model2 = DummyRegressor(strategy='constant', constant=-0.5)
    #model = SGDRegressor(max_iter=300)
    model = Ridge(alpha=0.1)
    bl_mse1 = cross_val_score(bl_model1, Xt_tfidf, Yt, scoring='neg_mean_squared_error', cv=5)
    #bl_mse2 = cross_val_score(bl_model2, Xt_tfidf, Yt, scoring='neg_mean_squared_error', cv=5)
    mse = cross_val_score(model, Xt_tfidf, Yt, scoring='neg_mean_squared_error', cv=5)
    print(mse.mean())
    print(bl_mse1.mean())
    #print(bl_mse2.mean())

-47.69738133910281
-48.01679006767749


In [53]:
def read_data_2(sample_name, train_size=0.8, seed=None):
    df = pd.read_csv(PATH_REVSENTS % sample_name)[['pt_review', 'polarity', 'votes_up', 'nlp_words', 'raw_chars']].reset_index(drop=True)
    scaler = RobustScaler()
    df['pt_review_scaled'] = scaler.fit_transform(np.array(df['pt_review']).reshape(-1, 1))
    train, validation = train_test_split(df, train_size=train_size, random_state=seed)
    return train, validation

In [64]:
train, validation = read_data_2('eng_any_any_100000')
Xt = train[['polarity', 'votes_up', 'nlp_words', 'raw_chars']].values.tolist()
Yt = train['pt_review_scaled'].tolist()
Xv = validation[['polarity', 'votes_up', 'nlp_words', 'raw_chars']].values.tolist()
Yv = validation['pt_review_scaled'].tolist()

In [99]:
bl = DummyRegressor(strategy='mean')
#model = SGDRegressor(max_iter=100, penalty='l2', alpha=0.1)
model = Ridge(alpha=0.1)
#model = Lasso(alpha=0.1)
bl_mse = cross_val_score(bl, Xt, Yt, scoring='neg_mean_squared_error', cv=5)
mse = cross_val_score(model, Xt, Yt, scoring='neg_mean_squared_error', cv=5)

In [100]:
print(bl_mse.mean())
print(mse.mean())

-50.919435471577266
-50.85524236756824
